In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time

def get_news_links_with_selenium(base_url):
    options = Options()
    options.headless = False  # 디버깅을 위해 headless 모드를 꺼둡니다.
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(base_url)
    links = set()  # 중복을 방지하기 위해 set을 사용합니다.
    i = 0

    while True:
        time.sleep(2)  # 페이지 로딩을 기다림
        # '더 보기' 버튼을 찾고 클릭
        try:
            # 페이지 하단으로 스크롤
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            load_more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "div#load-more button.button-fluid.cta-large"))
            )
            load_more_button.click()
            i += 1
            print(i)
            if i == 365:
                break
        except Exception as e:
            print(f"더 이상 '더 보기' 버튼을 찾을 수 없습니다. 에러: {e}")
            break
        
        # 매번 '더 보기' 버튼을 클릭한 후에 현재 페이지에서 링크를 수집합니다.
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        news_links = soup.find_all('a', {'data-description': 'title'})
        new_links = {link['href'] for link in news_links if 'href' in link.attrs}
        links.update(new_links)
    
    driver.quit()
    return list(links)

def crawl_news_page(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 타이틀 추출
    title = soup.find('title').text.strip()

    # 본문 추출
    content_div = soup.find('div', class_='wpb_wrapper')
    sections = content_div.find_all('section')
    content = '\n'.join([section.text.strip() for section in sections])
    
    return {
        'title': title,
        'content': content,
        'url': url
    }

base_url = 'https://www.bloomberg.co.kr/blog/category/news/'
news_links = get_news_links_with_selenium(base_url)
print(news_links)

# data = []

# for link in news_links:
#     news_data = crawl_news_page(link)
#     if news_data:
#         data.append(news_data)

# # Create a DataFrame from the collected data
# df = pd.DataFrame(data)
# print(df.head())


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [29]:
df

,title,date,content,url
0,오늘의 5가지 이슈: 긴축베팅 과도? ECB 7월 인상 - Bloomberg Pro...,2022.04.21,(블룸버그) — 인플레이션이 피크에 가까워지고 금리 인상 베팅이 과도하다는 진단이 ...,https://www.bloomberg.co.kr/blog/five-bond-rep...
1,오늘의 5가지 이슈: 美블루스윕? 사우디감산 - Bloomberg Professio...,2021.01.06,(블룸버그) — 미국 상원의 다수당을 결정지을 조지아주 결선투표가 진행되고 있는 가...,https://www.bloomberg.co.kr/blog/five-bluesweep/
2,"오늘의 5가지 이슈: OECD도 하향, 테일리스크 - Bloomberg Profes...",2022.06.09,(블룸버그) — 뉴욕증시는 인플레이션과 그에 따른 통화정책 긴축 우려 속에 3거래일...,https://www.bloomberg.co.kr/blog/oecd-tailrisk/
3,오늘의 5가지 이슈: 인플레 피크? 되살아난 75bp - Bloomberg Prof...,2022.05.11,(블룸버그) — 뉴욕증시는 최근 며칠 사이에 1.7조 달러 가량 시가총액이 날아간 ...,https://www.bloomberg.co.kr/blog/five-inflatio...
4,"오늘의 5가지 이슈: 어닝시즌 기대, 시진핑 경고 - Bloomberg Profes...",2022.01.18,(블룸버그) — 중국을 비롯해 세계 주요 경제가 팬데믹발 침체에서 강하게 반등한 이...,https://www.bloomberg.co.kr/blog/five-earnings...
5,"오늘의 5가지 이슈: 옐런방중, 연준 의사록 - Bloomberg Professio...",2023.07.06,(블룸버그) — 미-중간 기술 전쟁이 가열되고 있는 가운데 옐런 미 재무장관이 양국...,https://www.bloomberg.co.kr/blog/five-yellen-c...
6,"오늘의 5가지 이슈: 퍼스트 리퍼블릭 불안, 연준인상 불확실 - Bloomberg ...",2023.04.26,(블룸버그) — 퍼스트 리퍼블릭 은행이 어닝쇼크로 다시 투자자 불안을 키우면서 뉴욕...,https://www.bloomberg.co.kr/blog/five-first-re...
7,"오늘 5가지 이슈: 美물가우려 일축, 日여행중단 - Bloomberg Profess...",2021.05.25,일부 연준 인사들이 미국내 경제활동이 재개되는 가운데 물가 압력의 급등세가 지속될 ...,https://www.bloomberg.co.kr/blog/five-things-t...
8,"오늘의 5가지 이슈: ‘연준은 고릴라’, 스웨덴 인하 - Bloomberg Prof...",2024.05.09,(블룸버그) — 뉴욕증시는 인텔 악재와 연준 인사의 매파적 발언 속에 5거래일만에 ...,https://www.bloomberg.co.kr/blog/fed-ecb-swede...
9,"오늘의 5가지 이슈: 침체 우려에 내년 금리인하 예상, 유가 하락 - Bloombe...",2022.07.06,,https://www.bloomberg.co.kr/blog/five-july-6/


In [1]:
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd

def get_news_links(base_url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    i = 0
    links = []
    page_number = 1
    while True:
        url = f"{base_url}?page={page_number}"
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
            break

        soup = BeautifulSoup(response.text, 'html.parser')
        news_links = soup.find_all('a', {'data-description': 'title'})
        links.extend([link['href'] for link in news_links if 'href' in link.attrs])
        print(page_number)
        # Check if there is a "load more" button
        load_more_div = soup.find('div', id='load-more')
        if load_more_div and '더 보기' in load_more_div.text:
            page_number += 1
            time.sleep(random.uniform(1, 1.1))  # To avoid overloading the server
            if page_number == 3:
                break
        else:
            break
    return links

def crawl_news_page(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 타이틀 추출
    title = soup.find('title').text.strip()

    # 본문 추출
    content_div = soup.find('div', class_='wpb_wrapper')
    sections = content_div.find_all('section')
    content = '\n'.join([section.text.strip() for section in sections])
    
    return {
        'title': title,
        'content': content,
        'url': url
    }

base_url = 'https://www.bloomberg.co.kr/blog/category/news/'
news_links = get_news_links(base_url)
print(news_links, base_url)

data = []

for link in news_links:
    news_data = crawl_news_page(link)
    if news_data:
        data.append(news_data)

# Create a DataFrame from the collected data
df = pd.DataFrame(data)
print(df.head())


1
2
['https://www.bloomberg.co.kr/blog/five-jolts-pimco-bond/', 'https://www.bloomberg.co.kr/blog/five-ism-manufacturing-trump-bond/', 'https://www.bloomberg.co.kr/blog/five-april-pce-opec/', 'https://www.bloomberg.co.kr/blog/five-trump-guilty-cre/', 'https://www.bloomberg.co.kr/blog/five-private-credit-fed-beigebook/', 'https://www.bloomberg.co.kr/blog/five-t1-fed-hike/', 'https://www.bloomberg.co.kr/blog/five-things-to-know-155/', 'https://www.bloomberg.co.kr/blog/five-pce-korea-china-japan/', 'https://www.bloomberg.co.kr/blog/five-fed-cut-dec-cmbs/', 'https://www.bloomberg.co.kr/blog/five-fed-minutes-boe-2/', 'https://www.bloomberg.co.kr/blog/five-jolts-pimco-bond/', 'https://www.bloomberg.co.kr/blog/five-ism-manufacturing-trump-bond/', 'https://www.bloomberg.co.kr/blog/five-april-pce-opec/', 'https://www.bloomberg.co.kr/blog/five-trump-guilty-cre/', 'https://www.bloomberg.co.kr/blog/five-private-credit-fed-beigebook/', 'https://www.bloomberg.co.kr/blog/five-t1-fed-hike/', 'https://

In [4]:
import concurrent.futures
import pandas as pd
flag = 0
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    future_to_url = {executor.submit(crawl_news_page, link): link for link in final_list}
    for future in concurrent.futures.as_completed(future_to_url):
        link = future_to_url[future]
        try:
            news_data = future.result()
            if news_data:
                data.append(news_data)
        except Exception as e:
            print(f"{link}에서 데이터를 크롤링하는 중 오류 발생: {e}")

        if flag%10 == 0:
            print(flag)
        flag +=1

# 수집된 데이터로 DataFrame 생성
df = pd.DataFrame(data)
print(df.head())

0
10
https://www.mk.co.kr/news/it/7278739에서 데이터를 크롤링하는 중 오류 발생: 'NoneType' object has no attribute 'find'
https://www.mk.co.kr/news/economy/7279702에서 데이터를 크롤링하는 중 오류 발생: 'NoneType' object has no attribute 'find'
20
30
https://www.mk.co.kr/news/economy/7279763에서 데이터를 크롤링하는 중 오류 발생: 'NoneType' object has no attribute 'find'
40
50
60
70
80
https://www.mk.co.kr/news/business/7285926에서 데이터를 크롤링하는 중 오류 발생: 'NoneType' object has no attribute 'find'
https://www.mk.co.kr/news/economy/7288452에서 데이터를 크롤링하는 중 오류 발생: 'NoneType' object has no attribute 'find'
90
100
110
120
https://www.mk.co.kr/news/economy/7294996에서 데이터를 크롤링하는 중 오류 발생: 'NoneType' object has no attribute 'find'
130
https://www.mk.co.kr/news/it/7297435에서 데이터를 크롤링하는 중 오류 발생: 'NoneType' object has no attribute 'find'
140
150
160
170
https://www.mk.co.kr/news/economy/7304182에서 데이터를 크롤링하는 중 오류 발생: 'NoneType' object has no attribute 'find'
180
https://www.mk.co.kr/news/economy/7304221에서 데이터를 크롤링하는 중 오류 발생: 'NoneType' object has no attribu

In [7]:
df.to_excel('mail.xlsx')
df.to_csv('mail.csv')

In [11]:
df

,title,date,content,url
0,"외국인, 실적부진 대형주로 몰린다…왜? - 매일경제",2016-03-25 11:22:22,사진 확대최근 외국계 자금이 국내 증시에 든든한 버팀목으로 작용하고 있다. 코스피 ...,https://www.mk.co.kr/news/stock/7276327
1,"코스피, 2.16p(0.11%) 내린 1983.81 마감 - 매일경제",2016-03-25 15:03:04,"[디지털뉴스국 김경택 기자][ⓒ 매일경제 & mk.co.kr, 무단전재 및 재배포 금지]",https://www.mk.co.kr/news/stock/7276705
2,"미적지근한 코스피, 3거래일 연속 하락…1980선 약보합 - 매일경제",2016-03-25 15:20:15,코스피가 장 내내 미적지근한 움직임을 보이며 1980선 약보합권에서 마감했다. 최근...,https://www.mk.co.kr/news/stock/7276747
3,주총꾼이 직업? 가업승계까지 - 매일경제,2016-03-25 15:55:15,"외제차 타고 증권사 온 노신사 ""앞으로는 며느리에게 돈줘라""여의도 소재 한 증권사 ...",https://www.mk.co.kr/news/stock/7276850
4,[콕콕재테크] 안전벨트 맨 ELS - 매일경제,2016-03-25 16:00:18,원금손실 확 줄인 신상품 잇단 출시…ISA계좌에 담으면 세혜택 일석이조◆ 금융투자 ...,https://www.mk.co.kr/news/special-edition/7276884
...,...,...,...,...
25573,"코스피 마감, 2,689.5P (1.03%↑) LG에너지솔루션, 삼성전자등 상승세 ...",2024-06-05 15:35:40,"05일 아침에 상승세로 출발했던 코스피 시장은 장중 한때 2,667.63포인트까지 ...",https://www.mk.co.kr/news/stock/11034119
25574,애플·인텔도 시선집중…반도체 공정 세계 첫 상용화에 매수버튼 [주식 초고수는 지금]...,2024-06-05 14:45:51,"[수익률 상위 1% 투자자 오후 1시 30분 거래동향]주성엔지니어링, 3-5족 반도...",https://www.mk.co.kr/news/stock/11034048
25575,"“찐바닥 도달, 이번 달이 줍줍 기회”…단기간 20% 오를 수 있다는 이 종목, 믿...",2024-06-05 16:23:37,주가 변동 미미하지만…‘찐바닥’ 저점 매수 기회6%대 높은 배당수익률에 주주환원 강...,https://www.mk.co.kr/news/stock/11034188
25576,배반 없는 배터리·반도체…코스피 웃었다 - 매일경제,2024-06-05 18:04:44,엔비디아 HBM 납품 시사에삼성전자 하루새 2.8% 올라LG엔솔·삼성SDI도 상승세...,https://www.mk.co.kr/news/stock/11034403


In [58]:
import pandas as pd

# 데이터프레임을 엑셀 파일로 저장
df.to_excel('news_data_10_0529fdsaf1028ads.xlsx', index=False)


In [126]:
headers = {"User-Agent":"Mozilla/5.0"}
url = f'https://www.mk.co.kr/search?word=%EC%BD%94%EC%8A%A4%ED%94%BC&page=6'
print(url)
headers = {"User-Agent":"Mozilla/5.0"}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'lxml')
print(soup)

TypeError: WebDriver.__init__() got an unexpected keyword argument 'executable_path'

In [160]:
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd

def get_news_links(base_url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    
    links = []
    page_number = 1
    while True:
        url = f"{base_url}&page={page_number}"
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
            break

        soup = BeautifulSoup(response.text, 'html.parser')
        news_divs = soup.find_all('div', class_='txt_wrap')
        
        for div in news_divs:
            link_tag = div.find('a', href=True)
            if link_tag:
                links.append(link_tag['href'])
        try:
            page_number += 1
            time.sleep(random.uniform(1, 1.1))  # To avoid overloading the server
            if page_number == 10000:
                break
        except:
            break
        print(page_number)
    
    return links

def crawl_news_page(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 타이틀 추출
    title = soup.find('title').text.strip()

    # 본문 추출
    content = ''
    article_body = soup.find('div', class_='article-body', id='articletxt')
    if article_body:
        # 모든 텍스트 추출
        for element in article_body.descendants:
            if element.name == 'br':
                content += '\n'
            elif element.string:
                content += element.string.strip() + ' '
    
    return {
        'title': title,
        'content': content.strip(),
        'url': url
    }

base_url = 'https://search.hankyung.com/search/news?query=%EC%BD%94%EC%8A%A4%ED%94%BC'
news_links = get_news_links(base_url)
print(news_links, base_url)


2
3
4
['https://www.hankyung.com/article/202405291663a', 'https://www.hankyung.com/article/2024052920111', 'https://www.hankyung.com/article/202405291690Y', 'https://www.hankyung.com/article/202405291680L', 'https://www.hankyung.com/article/202405291648Y', 'https://www.hankyung.com/article/202405291508i', 'https://www.hankyung.com/article/2024052916235', 'https://www.hankyung.com/article/2024052915326', 'https://www.hankyung.com/article/202405291488Y', 'https://www.hankyung.com/article/2024052914655', 'https://www.hankyung.com/article/202405290236i', 'https://www.hankyung.com/article/202405287669a', 'https://www.hankyung.com/article/202405290336i', 'https://www.hankyung.com/article/202405290804Y', 'https://www.hankyung.com/article/2024052907566', 'https://www.hankyung.com/article/2024052907375', 'https://www.hankyung.com/article/2024052905725', 'https://www.hankyung.com/article/2024052905575', 'https://www.hankyung.com/article/202405290490L', 'https://www.hankyung.com/article/202405290

In [1]:
import requests
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed

def fetch_page_links(page_number, base_url, headers):
    url = f"{base_url}&page={page_number}"
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve the page {page_number}. Status code: {response.status_code}")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')
    news_divs = soup.find_all('div', class_='txt_wrap')

    links = []
    for div in news_divs:
        link_tag = div.find('a', href=True)
        if link_tag:
            links.append(link_tag['href'])
    
    return links

def get_news_links(base_url, total_pages, chunk_size=100):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    
    all_links = []
    
    for start_page in range(1, total_pages + 1, chunk_size):
        end_page = min(start_page + chunk_size - 1, total_pages)
        links = []
        
        with ThreadPoolExecutor(max_workers=10) as executor:
            future_to_page = {executor.submit(fetch_page_links, page, base_url, headers): page for page in range(start_page, end_page + 1)}
            
            for future in as_completed(future_to_page):
                page_number = future_to_page[future]
                try:
                    page_links = future.result()
                    links.extend(page_links)
                except Exception as exc:
                    print(f"Page {page_number} generated an exception: {exc}")
        
        all_links.extend(links)
        print(f"Collected {len(links)} links from pages {start_page} to {end_page}. Total links collected so far: {len(all_links)}")
    
    return all_links

# 예제 실행
base_url = 'https://search.hankyung.com/search/news?query=%EC%BD%94%EC%8A%A4%ED%94%BC'
total_pages = 10000  # 수집할 총 페이지 수
news_links = get_news_links(base_url, total_pages)
print(f"Total links collected: {len(news_links)}")


Collected 1000 links from pages 1 to 100. Total links collected so far: 1000
Collected 1000 links from pages 101 to 200. Total links collected so far: 2000
Collected 1000 links from pages 201 to 300. Total links collected so far: 3000
Collected 1000 links from pages 301 to 400. Total links collected so far: 4000
Collected 1000 links from pages 401 to 500. Total links collected so far: 5000


KeyboardInterrupt: 

In [2]:
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd



def get_news_links(base_url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    
    links = []; flag = 0
    page_number = 1
    while True:
        url = f"{base_url}?page={page_number}"
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
            break

        soup = BeautifulSoup(response.text, 'html.parser')
        news_divs = soup.find_all('div', class_='thumb')
        flag +=1
        
        for div in news_divs:
            link_tag = div.find('a', href=True)
            if link_tag:
                links.append(link_tag['href'])
            else:
                print(" 에러발생 ")
        try:
            page_number += 1
            time.sleep(random.uniform(1.0, 1.1))  # To avoid overloading the server
            if page_number == 450:
                break
        except:
            print(" 에러발생 ")
            break
        if flag&10 == 0:
            print(f'진행상황 = {page_number}')
    
    return links

def crawl_news_page(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    # 타이틀 추출
    title = soup.find('title').text.strip()

    # 본문 추출
    content = ''
    article_body = soup.find('div', class_='article-body', id='articletxt')
    if article_body:
        # 모든 텍스트 추출
        for element in article_body.descendants:
            if element.name == 'br':
                content += '\n'
            elif element.string:
                content += element.string.strip() + ' '
    
    return {
        'title': title,
        'content': content.strip(),
        'url': url
    }

base_url = 'https://www.hankyung.com/tag/%EC%BD%94%EC%8A%A4%ED%94%BC'
news_links = get_news_links(base_url)
print(news_links, base_url)

진행상황 = 2


ConnectionError: ('Connection aborted.', ConnectionResetError(10054, '현재 연결은 원격 호스트에 의해 강제로 끊겼습니다', None, 10054, None))

In [16]:
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd
from fake_useragent import UserAgent

# fake_useragent 인스턴스 생성
ua = UserAgent()

def get_news_links(base_url):
    headers = {
        'User-Agent': ua.random}
    
    links = []; flag = 0
    page_number = 1
    while True:
        url = f"{base_url}?page={page_number}"
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
            break

        soup = BeautifulSoup(response.text, 'html.parser')
        news_divs = soup.find_all('div', class_='thumb')
        flag +=1
        
        for div in news_divs:
            link_tag = div.find('a', href=True)
            if link_tag:
                links.append(link_tag['href'])
            else:
                print(" 에러발생 ")
        page_number += 1
        time.sleep(random.uniform(0.1, 0.2))  # To avoid overloading the server
        if page_number == 450:
            break
        print(f'진행상황 = {page_number}')
    
    return links

def crawl_news_page(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    # 타이틀 추출
    title = soup.find('title').text.strip()

    # 본문 추출
    content = ''
    article_body = soup.find('div', class_='article-body', id='articletxt')
    if article_body:
        # 모든 텍스트 추출
        for element in article_body.descendants:
            if element.name == 'br':
                content += '\n'
            elif element.string:
                content += element.string.strip() + ' '
    
    return {
        'title': title,
        'content': content.strip(),
        'url': url
    }

base_url = 'https://www.hankyung.com/tag/%EC%BD%94%EC%8A%A4%ED%94%BC'
news_links = get_news_links(base_url)
print(news_links, base_url)

진행상황 = 2
진행상황 = 3
진행상황 = 4
진행상황 = 5
진행상황 = 6
진행상황 = 7
진행상황 = 8
진행상황 = 9
진행상황 = 10
진행상황 = 11
진행상황 = 12
진행상황 = 13
진행상황 = 14
진행상황 = 15
진행상황 = 16
진행상황 = 17
진행상황 = 18
진행상황 = 19
진행상황 = 20
진행상황 = 21
진행상황 = 22
진행상황 = 23
진행상황 = 24
진행상황 = 25
진행상황 = 26
진행상황 = 27
진행상황 = 28
진행상황 = 29
진행상황 = 30
진행상황 = 31
진행상황 = 32
진행상황 = 33
진행상황 = 34
진행상황 = 35
진행상황 = 36
진행상황 = 37
진행상황 = 38
진행상황 = 39
진행상황 = 40
진행상황 = 41
진행상황 = 42
진행상황 = 43
진행상황 = 44
진행상황 = 45
진행상황 = 46
진행상황 = 47
진행상황 = 48
진행상황 = 49
진행상황 = 50
진행상황 = 51
진행상황 = 52
진행상황 = 53
진행상황 = 54
진행상황 = 55
진행상황 = 56
진행상황 = 57
진행상황 = 58
진행상황 = 59
진행상황 = 60
진행상황 = 61
진행상황 = 62
진행상황 = 63
진행상황 = 64
진행상황 = 65
진행상황 = 66
진행상황 = 67
진행상황 = 68
진행상황 = 69
진행상황 = 70
진행상황 = 71
진행상황 = 72
진행상황 = 73
진행상황 = 74
진행상황 = 75
진행상황 = 76
진행상황 = 77
진행상황 = 78
진행상황 = 79
진행상황 = 80
진행상황 = 81
진행상황 = 82
진행상황 = 83
진행상황 = 84
진행상황 = 85
진행상황 = 86
진행상황 = 87
진행상황 = 88
진행상황 = 89
진행상황 = 90
진행상황 = 91
진행상황 = 92
진행상황 = 93
진행상황 = 94
진행상황 = 95
진행상황 = 96
진행상황 = 97
진행상황 = 98
진행상황 = 99
진행상황 = 100
진행상황 = 101
진행상황 =

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, '현재 연결은 원격 호스트에 의해 강제로 끊겼습니다', None, 10054, None))

In [14]:
import requests
from bs4 import BeautifulSoup
import time
import random
import pandas as pd

def get_news_links(base_url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36'}
    
    links = []; flag = 0
    page_number = 1
    while True:
        url = f"{base_url}?page={page_number}"
        response = requests.get(url, headers=headers)
        if response.status_code != 200:
            print(f"Failed to retrieve the page. Status code: {response.status_code}")
            break

        soup = BeautifulSoup(response.text, 'html.parser')
        news_divs = soup.find_all('div', class_='thumb')
        flag +=1
        
        for div in news_divs:
            link_tag = div.find('a', href=True)
            if link_tag:
                links.append(link_tag['href'])
            else:
                print(" 에러발생 ")
        page_number += 1
        time.sleep(random.uniform(0.5, 0.6))  # To avoid overloading the server
        if page_number == 450:
            break
        print(f'진행상황 = {page_number}')
    
    return links

def crawl_news_page(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    # 타이틀 추출
    title = soup.find('title').text.strip()

    # 본문 추출
    content = ''
    article_body = soup.find('div', class_='article-body', id='articletxt')
    if article_body:
        # 모든 텍스트 추출
        for element in article_body.descendants:
            if element.name == 'br':
                content += '\n'
            elif element.string:
                content += element.string.strip() + ' '
    
    return {
        'title': title,
        'content': content.strip(),
        'url': url
    }

base_url = 'https://www.hankyung.com/tag/%EC%BD%94%EC%8A%A4%ED%94%BC'
news_links = get_news_links(base_url)
print(news_links, base_url)

진행상황 = 2
진행상황 = 3
진행상황 = 4
진행상황 = 5
진행상황 = 6
진행상황 = 7
진행상황 = 8
진행상황 = 9
진행상황 = 10
진행상황 = 11
진행상황 = 12
진행상황 = 13
진행상황 = 14
진행상황 = 15
진행상황 = 16
진행상황 = 17
진행상황 = 18
진행상황 = 19
진행상황 = 20
진행상황 = 21
진행상황 = 22
진행상황 = 23
진행상황 = 24
진행상황 = 25
진행상황 = 26
진행상황 = 27
진행상황 = 28
진행상황 = 29
진행상황 = 30
진행상황 = 31
진행상황 = 32
진행상황 = 33
진행상황 = 34
진행상황 = 35
진행상황 = 36
진행상황 = 37
진행상황 = 38
진행상황 = 39
진행상황 = 40
진행상황 = 41
진행상황 = 42
진행상황 = 43
진행상황 = 44
진행상황 = 45
진행상황 = 46
진행상황 = 47
진행상황 = 48
진행상황 = 49
진행상황 = 50
진행상황 = 51
진행상황 = 52
진행상황 = 53
진행상황 = 54
진행상황 = 55
진행상황 = 56
진행상황 = 57
진행상황 = 58
진행상황 = 59
진행상황 = 60
진행상황 = 61
진행상황 = 62
진행상황 = 63
진행상황 = 64
진행상황 = 65
진행상황 = 66
진행상황 = 67
진행상황 = 68
진행상황 = 69
진행상황 = 70
진행상황 = 71
진행상황 = 72
진행상황 = 73
진행상황 = 74
진행상황 = 75
진행상황 = 76
진행상황 = 77
진행상황 = 78
진행상황 = 79
진행상황 = 80
진행상황 = 81
진행상황 = 82
진행상황 = 83
진행상황 = 84
진행상황 = 85
진행상황 = 86
진행상황 = 87
진행상황 = 88
진행상황 = 89
진행상황 = 90
진행상황 = 91
진행상황 = 92
진행상황 = 93
진행상황 = 94
진행상황 = 95


ConnectionError: ('Connection aborted.', ConnectionResetError(10054, '현재 연결은 원격 호스트에 의해 강제로 끊겼습니다', None, 10054, None))

In [2]:
import requests
from bs4 import BeautifulSoup
import time
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
from fake_useragent import UserAgent

# fake_useragent 인스턴스 생성
ua = UserAgent()

def fetch_page_links(page_number, base_url):
    headers = {'User-Agent': ua.random}
    url = f"{base_url}?page={page_number}"
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve the page {page_number}. Status code: {response.status_code}")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')
    news_divs = soup.find_all('div', class_='thumb')

    links = []
    for div in news_divs:
        link_tag = div.find('a', href=True)
        if link_tag:
            links.append(link_tag['href'])

            
    
    # 요청 간 지연 시간을 추가
    time.sleep(random.uniform(0.1, 0.2))
    
    return links

def get_news_links(base_url, total_pages, chunk_size=10):
    all_links = []
    
    for start_page in range(1, total_pages + 1, chunk_size):
        end_page = min(start_page + chunk_size - 1, total_pages)
        links = []
        
        with ThreadPoolExecutor(max_workers=5) as executor:
            future_to_page = {
                executor.submit(fetch_page_links, page, base_url): page
                for page in range(start_page, end_page + 1)
            }
            
            for future in as_completed(future_to_page):
                page_number = future_to_page[future]
                try:
                    page_links = future.result()
                    links.extend(page_links)
                except Exception as exc:
                    print(f"Page {page_number} generated an exception: {exc}")
        
        all_links.extend(links)
        print(f"Collected {len(links)} links from pages {start_page} to {end_page}. Total links collected so far: {len(all_links)}")
    
    return all_links

# 예제 실행
base_url = 'https://www.hankyung.com/tag/%EC%BD%94%EC%8A%A4%ED%94%BC'
total_pages = 450 # 수집할 총 페이지 수
news_links = get_news_links(base_url, total_pages)
print(f"Total links collected: {len(news_links)}")


Collected 268 links from pages 1 to 10. Total links collected so far: 268
Collected 264 links from pages 11 to 20. Total links collected so far: 532
Collected 270 links from pages 21 to 30. Total links collected so far: 802
Collected 269 links from pages 31 to 40. Total links collected so far: 1071
Collected 256 links from pages 41 to 50. Total links collected so far: 1327
Collected 254 links from pages 51 to 60. Total links collected so far: 1581
Collected 260 links from pages 61 to 70. Total links collected so far: 1841
Page 72 generated an exception: ('Connection aborted.', ConnectionResetError(10054, '현재 연결은 원격 호스트에 의해 강제로 끊겼습니다', None, 10054, None))
Collected 216 links from pages 71 to 80. Total links collected so far: 2057
Collected 255 links from pages 81 to 90. Total links collected so far: 2312
Page 94 generated an exception: ('Connection aborted.', ConnectionResetError(10054, '현재 연결은 원격 호스트에 의해 강제로 끊겼습니다', None, 10054, None))
Collected 246 links from pages 91 to 100. Total li

In [4]:
with open('my_list.txt', 'w') as file:
    for item in news_links:
        file.write(item + '\n')

In [5]:
import concurrent.futures
import pandas as pd

def crawl_news_page(url):
    headers = {'User-Agent': ua.random}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    # 타이틀 추출
    title = soup.find('title').text.strip()

    # 본문 추출
    content = ''
    article_body = soup.find('div', class_='article-body', id='articletxt')
    if article_body:
        # 모든 텍스트 추출
        for element in article_body.descendants:
            if element.name == 'br':
                content += '\n'
            elif element.string:
                content += element.string.strip() + ' '
    
    return {
        'title': title,
        'content': content.strip(),
        'url': url
    }


flag = 0; data=[]
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    future_to_url = {executor.submit(crawl_news_page, link): link for link in news_links}
    for future in concurrent.futures.as_completed(future_to_url):
        link = future_to_url[future]
        try:
            news_data = future.result()
            if news_data:
                data.append(news_data)
        except Exception as e:
            print(f"{link}에서 데이터를 크롤링하는 중 오류 발생: {e}")

        if flag%10 == 0:
            print(flag)
        flag +=1

# 수집된 데이터로 DataFrame 생성
df = pd.DataFrame(data)
print(df.head())

0
10
20
https://www.hankyung.com/article/2024052093786에서 데이터를 크롤링하는 중 오류 발생: ('Connection aborted.', ConnectionResetError(10054, '현재 연결은 원격 호스트에 의해 강제로 끊겼습니다', None, 10054, None))
30
40
50
60
70
80
https://www.hankyung.com/article/2024052901636에서 데이터를 크롤링하는 중 오류 발생: ('Connection aborted.', ConnectionResetError(10054, '현재 연결은 원격 호스트에 의해 강제로 끊겼습니다', None, 10054, None))
90
100
110
120
130
140
https://www.hankyung.com/article/2024040861146에서 데이터를 크롤링하는 중 오류 발생: ('Connection aborted.', ConnectionResetError(10054, '현재 연결은 원격 호스트에 의해 강제로 끊겼습니다', None, 10054, None))
https://www.hankyung.com/article/2024040523446에서 데이터를 크롤링하는 중 오류 발생: ('Connection aborted.', ConnectionResetError(10054, '현재 연결은 원격 호스트에 의해 강제로 끊겼습니다', None, 10054, None))
150
160
170
180
190
200
210
220
230
https://www.hankyung.com/article/202405303843Y에서 데이터를 크롤링하는 중 오류 발생: ('Connection aborted.', ConnectionResetError(10054, '현재 연결은 원격 호스트에 의해 강제로 끊겼습니다', None, 10054, None))
240
250
260
270
280
https://www.hankyung.com/article/20

In [7]:
df.to_excel("hangyoung_0530.xlsx")

In [3]:
import pandas as pd
df = pd.read_csv('C:/Users/82107/Desktop/동아리/news_data_10_0529.csv')

In [12]:
df.to_excel("news_0529.xlsx")

In [11]:
from datetime import datetime, timedelta
num_rows = df.shape[0]
start_date = datetime(2024, 5, 29)
dates = [start_date - timedelta(days=i) for i in range(num_rows)]

# Add the 'date' column to the DataFrame
df['date'] = dates
df.head()


,title,content,url,date
0,"오늘의 5가지 이슈: 美 T+1 시험대, 연준 인상 발언 - Bloomberg Pr...",서은경 기자\n\n(블룸버그) — 미국 증시의 결제주기가 현지시간 28일부터 기존 ...,https://www.bloomberg.co.kr/blog/five-t1-fed-h...,2024-05-29
1,"오늘 5가지 이슈: ECB 7월 인하기대, 금값 상승 - Bloomberg Prof...",유럽증시가 유럽중앙은행(ECB)의 6월과 7월 연속 금리 인하 가능성을 시사한 프랑...,https://www.bloomberg.co.kr/blog/five-things-t...,2024-05-28
2,오늘의 5가지 이슈: 美PCE 둔화? 韓·中·日 정상회의 - Bloomberg Pr...,(블룸버그) — 연준이 기저 인플레이션 지표로 주시하는 미국 근원 개인소비지출(PC...,https://www.bloomberg.co.kr/blog/five-pce-kore...,2024-05-27
3,"오늘의 5가지 이슈: 연준첫인하 12월, 美CMBS 손실 - Bloomberg Pr...",(블룸버그) — 이달 초 미국의 비즈니스 활동이 서비스업 덕분에 2년 만에 가장 빠...,https://www.bloomberg.co.kr/blog/five-fed-cut-...,2024-05-26
4,"오늘의 5가지 이슈: 매파 의사록, 英6월 인하 의심 - Bloomberg Prof...",(블룸버그) — 간밤 공개된 연준 의사록에서 “많은” 연준 위원들이 통화정책의 제약...,https://www.bloomberg.co.kr/blog/five-fed-minu...,2024-05-25


In [110]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
import traceback

def get_news_links_with_selenium(base_url):
    options = Options()
    options.headless = False  # 디버깅을 위해 headless 모드를 꺼둡니다.
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(base_url)

    links = set()  # 중복을 방지하기 위해 set을 사용합니다.
    i = 0
    max_attempts = 99
    load_timeout = 30  # 로딩 타임아웃을 10초로 설정

    while True:
        time.sleep(1)  # 페이지 로딩을 기다림
        # 페이지 하단으로 스크롤

        try:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(0.3)  # 스크롤 후 약간 대기
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1)
            # 로딩 중 상태 확인
            WebDriverWait(driver, load_timeout).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "button[data-btn-more]"))
            )
            
            load_more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-btn-more]"))
            )
            load_more_button.click()
            i += 1
            print(i)
            if i == max_attempts:
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                news_links = soup.find_all('a', class_='news_item')
                new_links = {link['href'] for link in news_links if 'href' in link.attrs}
                links.update(new_links)
                break
            
        except Exception as e:
            print(f"더 이상 '더 보기' 버튼을 찾을 수 없습니다. 에러: {e}")
            traceback.print_exc()
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            news_links = soup.find_all('a', class_='news_item')
            new_links = {link['href'] for link in news_links if 'href' in link.attrs}
            links.update(new_links)
    
    driver.quit()
    return list(links)

def crawl_news_page(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 타이틀 추출
    title = soup.find('title').text.strip()

    # 날짜 추출
    date_div = soup.find('div', class_='article__date')
    date = date_div.get_text(strip=True) if date_div else 'N/A'

    # 본문 추출
    content_div = soup.find('div', class_='art_txt')
    paragraphs = content_div.find_all('p') if content_div else []
    content = '\n'.join([paragraph.text.strip() for paragraph in paragraphs])
    
    return {
        'title': title,
        'date': date,
        'content': content,
        'url': url
    }

base_url = 'https://www.mk.co.kr/search/news?word=%EC%BD%94%EC%8A%A4%ED%94%BC&dateType=1year&startDate=2016-01-01&endDate=2016-10-27'
news_links2 = get_news_links_with_selenium(base_url)
print(news_links2)

data = []

# for link in news_links:
#     news_data = crawl_news_page(link)
#     if news_data:
#         data.append(news_data)

# # Create a DataFrame from the collected data
# df = pd.DataFrame(data)
# print(df.head())


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
['https://www.mk.co.kr/news/stock/7406575', 'https://www.mk.co.kr/news/stock/7402940', 'https://www.mk.co.kr/news/stock/7482006', 'https://www.mk.co.kr/news/stock/7442304', 'https://www.mk.co.kr/news/stock/7396817', 'https://www.mk.co.kr/news/stock/7360810', 'https://www.mk.co.kr/news/stock/7373339', 'https://www.mk.co.kr/news/stock/7527347', 'https://www.mk.co.kr/news/stock/7460119', 'https://www.mk.co.kr/news/stock/7415806', 'https://www.mk.co.kr/news/stock/7446691', 'https://www.mk.co.kr/news/stock/7431620', 'https://www.mk.co.kr/news/stock/7351930', 'https://www.mk.co.kr/news/stock/7550810', 'https://www.mk.co.kr/news/economy/7423819', 'https://www.mk.co.kr/news/stock/7345345', 'https://www.mk.co.kr

In [111]:
news_links2.sort(key=lambda x: int(x.rstrip('/').split('/')[-1]))
news_links2

['https://www.mk.co.kr/news/stock/7276327',
 'https://www.mk.co.kr/news/stock/7276705',
 'https://www.mk.co.kr/news/stock/7276747',
 'https://www.mk.co.kr/news/stock/7276850',
 'https://www.mk.co.kr/news/special-edition/7276884',
 'https://www.mk.co.kr/news/stock/7276960',
 'https://www.mk.co.kr/news/stock/7278126',
 'https://www.mk.co.kr/news/stock/7278134',
 'https://www.mk.co.kr/news/stock/7278303',
 'https://www.mk.co.kr/news/stock/7278304',
 'https://www.mk.co.kr/news/stock/7278384',
 'https://www.mk.co.kr/news/stock/7278386',
 'https://www.mk.co.kr/news/it/7278739',
 'https://www.mk.co.kr/news/stock/7278758',
 'https://www.mk.co.kr/news/stock/7278816',
 'https://www.mk.co.kr/news/stock/7278967',
 'https://www.mk.co.kr/news/stock/7279006',
 'https://www.mk.co.kr/news/stock/7279098',
 'https://www.mk.co.kr/news/stock/7279487',
 'https://www.mk.co.kr/news/stock/7279562',
 'https://www.mk.co.kr/news/stock/7279574',
 'https://www.mk.co.kr/news/stock/7279618',
 'https://www.mk.co.kr/ne

In [112]:
final += news_links2
len(final)

27240

In [114]:
final = list(set(final))
len(final)

27171

In [124]:
file_path = "output.txt"
with open(file_path,"w") as file:
    for item in final:
        file.write(item + "\n")

In [120]:
final.sort(key=lambda x: int(x.rstrip('/').split('/')[-1]))
final

['https://www.mk.co.kr/news/stock/7276327',
 'https://www.mk.co.kr/news/stock/7276705',
 'https://www.mk.co.kr/news/stock/7276747',
 'https://www.mk.co.kr/news/stock/7276850',
 'https://www.mk.co.kr/news/special-edition/7276884',
 'https://www.mk.co.kr/news/stock/7276960',
 'https://www.mk.co.kr/news/stock/7278126',
 'https://www.mk.co.kr/news/stock/7278134',
 'https://www.mk.co.kr/news/stock/7278303',
 'https://www.mk.co.kr/news/stock/7278304',
 'https://www.mk.co.kr/news/stock/7278384',
 'https://www.mk.co.kr/news/stock/7278386',
 'https://www.mk.co.kr/news/it/7278739',
 'https://www.mk.co.kr/news/stock/7278758',
 'https://www.mk.co.kr/news/stock/7278816',
 'https://www.mk.co.kr/news/stock/7278967',
 'https://www.mk.co.kr/news/stock/7279006',
 'https://www.mk.co.kr/news/stock/7279098',
 'https://www.mk.co.kr/news/stock/7279487',
 'https://www.mk.co.kr/news/stock/7279562',
 'https://www.mk.co.kr/news/stock/7279574',
 'https://www.mk.co.kr/news/stock/7279618',
 'https://www.mk.co.kr/ne

In [2]:
# fake_useragent 인스턴스 생성
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
import time
import traceback
import requests


ua = UserAgent()


def crawl_news_page(url):
    headers = {'User-Agent': ua.random}
    # headers = {
    #     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 타이틀 추출
    title = soup.find('title').text.strip()

    # 날짜 추출
    date_dd = soup.find('dl', class_='registration').find('dd')
    date = date_dd.get_text(strip=True) if date_dd else 'N/A'

    # 본문 추출
    content_div = soup.find('div', class_='news_cnt_detail_wrap')
    content = content_div.get_text(strip=True) if content_div else 'N/A'
    
    return {
        'title': title,
        'date': date,
        'content': content,
        'url': url
    }


data = []
i=0
for link in final_list:
    news_data = crawl_news_page(link)
    i+=1
    if news_data:
        data.append(news_data)
    if i==10:
        break

# Create a DataFrame from the collected data
df = pd.DataFrame(data)
print(df.head())


                                    title                 date  \
0            외국인, 실적부진 대형주로 몰린다…왜? - 매일경제  2016-03-25 11:22:22   
1  코스피, 2.16p(0.11%) 내린 1983.81 마감 - 매일경제  2016-03-25 15:03:04   
2  미적지근한 코스피, 3거래일 연속 하락…1980선 약보합 - 매일경제  2016-03-25 15:20:15   
3                  주총꾼이 직업? 가업승계까지 - 매일경제  2016-03-25 15:55:15   
4               [콕콕재테크] 안전벨트 맨 ELS - 매일경제  2016-03-25 16:00:18   

                                             content  \
0  사진 확대최근 외국계 자금이 국내 증시에 든든한 버팀목으로 작용하고 있다. 코스피 ...   
1  [디지털뉴스국 김경택 기자][ⓒ 매일경제 & mk.co.kr, 무단전재 및 재배포 금지]   
2  코스피가 장 내내 미적지근한 움직임을 보이며 1980선 약보합권에서 마감했다. 최근...   
3  외제차 타고 증권사 온 노신사 "앞으로는 며느리에게 돈줘라"여의도 소재 한 증권사 ...   
4  원금손실 확 줄인 신상품 잇단 출시…ISA계좌에 담으면 세혜택 일석이조◆ 금융투자 ...   

                                                 url  
0            https://www.mk.co.kr/news/stock/7276327  
1            https://www.mk.co.kr/news/stock/7276705  
2            https://www.mk.co.kr/news/stock/7276747  
3            https://www.mk.co.kr/news/stock/7

In [3]:
df

,title,date,content,url
0,"외국인, 실적부진 대형주로 몰린다…왜? - 매일경제",2016-03-25 11:22:22,사진 확대최근 외국계 자금이 국내 증시에 든든한 버팀목으로 작용하고 있다. 코스피 ...,https://www.mk.co.kr/news/stock/7276327
1,"코스피, 2.16p(0.11%) 내린 1983.81 마감 - 매일경제",2016-03-25 15:03:04,"[디지털뉴스국 김경택 기자][ⓒ 매일경제 & mk.co.kr, 무단전재 및 재배포 금지]",https://www.mk.co.kr/news/stock/7276705
2,"미적지근한 코스피, 3거래일 연속 하락…1980선 약보합 - 매일경제",2016-03-25 15:20:15,코스피가 장 내내 미적지근한 움직임을 보이며 1980선 약보합권에서 마감했다. 최근...,https://www.mk.co.kr/news/stock/7276747
3,주총꾼이 직업? 가업승계까지 - 매일경제,2016-03-25 15:55:15,"외제차 타고 증권사 온 노신사 ""앞으로는 며느리에게 돈줘라""여의도 소재 한 증권사 ...",https://www.mk.co.kr/news/stock/7276850
4,[콕콕재테크] 안전벨트 맨 ELS - 매일경제,2016-03-25 16:00:18,원금손실 확 줄인 신상품 잇단 출시…ISA계좌에 담으면 세혜택 일석이조◆ 금융투자 ...,https://www.mk.co.kr/news/special-edition/7276884
5,"[장외주식시황] 레이언스, 수요예측 마지막 날 ↑… 에이프로젠, 해외판권 계약 소식...",2016-03-25 16:20:28,"코스피 지수가 기관이 순매수에 나섰으나, 외국인이 12거래일 만에 순매도로 돌아서며...",https://www.mk.co.kr/news/stock/7276960
6,국내주식형 펀드 5개중 1개만 약세장 이겼다 - 매일경제,2016-03-27 15:19:32,주식시장이 숨고르기에 들어간 지난 6개월 동안 시장수익률을 웃돈 국내주식형 펀드가 ...,https://www.mk.co.kr/news/stock/7278126
7,내달 로보어드바이저 활용한 ISA 등장...수익 얼마나 낼까 - 매일경제,2016-03-27 15:21:13,로보어드바이저(Robo-advisor)를 활용한 개인종합자산관리계좌(ISA) 상품이...,https://www.mk.co.kr/news/stock/7278134
8,약세장에 펀드 80% `울상`…수익률 코스피200 밑돌아 - 매일경제,2016-03-27 17:07:41,주식시장이 숨 고르기에 들어간 지난 6개월 동안 시장 수익률을 웃돈 국내 주식형 펀...,https://www.mk.co.kr/news/stock/7278303
9,동양파일 28일 공모청약 - 매일경제,2016-03-27 17:07:45,총 800만주…주당 공모가 1만원씨엠에스에듀·레이언스도 청약고강도 콘크리트파일(P...,https://www.mk.co.kr/news/stock/7278304


In [1]:
file_path = "output.txt"
with open(file_path, "r") as file:
    lines = file.readlines()

# 각 줄의 끝에 붙은 개행 문자를 제거합니다.
final_list = [line.strip() for line in lines]

print(len(final_list))

27171


In [4]:
import requests
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
response = requests.get('https://tradingeconomics.com/united-states/gdp-growth', headers=headers)
if response.status_code != 200:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")

soup = BeautifulSoup(response.text, 'html.parser')
print(soup)


<!DOCTYPE html>

<html>
<head id="ctl00_Head1"><meta charset="utf-8"/><title>
	United States GDP Growth Rate
</title><meta content="The Gross Domestic Product (GDP) in the United States expanded 1.30 percent in the first quarter of 2024 over the previous quarter. This page provides the latest reported value for - United States GDP Growth Rate - plus previous releases, historical high and low, short-term forecast and long-term prediction, economic calendar, survey consensus and news." id="metaDesc" name="description"/><meta content="United States,GDP Growth Rate,data,chart,actual,historical,values,calendar,forecast,graph,2023,2024,2025" id="metaKeyword" name="keywords"/><meta content="width=device-width,minimum-scale=1,initial-scale=1,maximum-scale=1" name="viewport"/><meta content="#333333" name="theme-color"/>
<link crossorigin="anonymous" href="https://cdnjs.cloudflare.com/ajax/libs/bootstrap/5.2.3/css/bootstrap.min.css" integrity="sha512-SbiR/eusphKoMVVXysTKG/7VseWii+Y3FdHrt0EpKgpT